In [ ]:
# ## 1. Import Library
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Setting untuk tampilan yang lebih baik
plt.style.use('default')
pd.set_option('display.max_columns', None)


In [ ]:
# ## 2. Load Data
df = pd.read_csv("../DATA/day.csv")
print(f"Data berhasil dimuat. Jumlah baris: {df.shape[0]}, Kolom: {df.shape[1]}")
print(f"Periode data: {df['dteday'].min()} - {df['dteday'].max()}")
print("\nTampilan 5 baris pertama:")
df.head()


In [ ]:
# ## 3. Bersihkan Data
# Hapus kolom yang tidak diperlukan
df_clean = df.drop(columns=["instant"]).copy()
df_clean['dteday'] = pd.to_datetime(df_clean['dteday'])

print(f"Data setelah dibersihkan: {df_clean.shape}")
print("\nInfo singkat dataset:")
print(df_clean.info())


In [ ]:
# ## 4. Lihat Hubungan Antar Variabel
# Hitung korelasi
corr_matrix = df_clean.corr()

print("Korelasi dengan target 'cnt' (jumlah penyewaan):")
cnt_corr = corr_matrix['cnt'].sort_values(ascending=False)

# Tampilkan yang paling berpengaruh
print("\nTop 5 variabel yang berkorelasi positif:")
for var, corr in cnt_corr.head(6).items():  # 6 karena cnt sendiri = 1.0
    if var != 'cnt':
        print(f"  {var}: {corr:.3f}")

print("\nTop 3 variabel yang berkorelasi negatif:")
for var, corr in cnt_corr.tail(3).items():
    print(f"  {var}: {corr:.3f}")


In [ ]:
# ## 5. Visualisasi Korelasi
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap="RdBu_r", fmt=".2f",
            center=0, square=True)
plt.title("Peta Korelasi Antar Variabel", size=14, pad=20)
plt.tight_layout()
plt.savefig('../OUTPUT/heatmap_korelasi.png', dpi=300, bbox_inches='tight')
plt.show()

print("Interpretasi:")
print("- Warna merah = korelasi positif (semakin merah, semakin kuat)")
print("- Warna biru = korelasi negatif (semakin biru, semakin kuat)")
print("- 'registered' dan 'casual' tidak akan dipakai karena cnt = registered + casual")


In [ ]:
# ## 6. Pilih Variabel untuk Model
# Variabel target (Y)
y = df_clean['cnt']

# Variabel input (X) - pilih yang masuk akal dan tidak ada data leakage
X_columns = ['temp', 'atemp', 'hum', 'windspeed', 'season', 'yr', 'mnth',
             'holiday', 'weekday', 'workingday', 'weathersit']

X = df_clean[X_columns]

print("Variabel yang dipilih:")
print(f"Target (Y): cnt (jumlah total penyewaan)")
print(f"Input (X): {len(X_columns)} variabel")
for i, col in enumerate(X_columns, 1):
    print(f"  {i}. {col}")

print(f"\nUkuran data:")
print(f"X: {X.shape[0]} baris, {X.shape[1]} kolom")
print(f"y: {y.shape[0]} data")


In [ ]:
# ## 7. Bagi Data Training dan Testing
# 80% untuk training, 20% untuk testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Pembagian data:")
print(f"Training: {len(X_train)} data ({len(X_train)/len(X)*100:.1f}%)")
print(f"Testing: {len(X_test)} data ({len(X_test)/len(X)*100:.1f}%)")


In [39]:
# ## 8. Latih Model
model = LinearRegression()
model.fit(X_train, y_train)

print("Model berhasil dilatih!")
print(f"\nParameter model:")
print(f"Intercept (konstanta): {model.intercept_:.1f}")

# Lihat koefisien masing-masing variabel
print(f"\nKoefisien variabel (pengaruh terhadap penyewaan):")
coef_df = pd.DataFrame({
    'Variabel': X.columns,
    'Koefisien': model.coef_
}).sort_values('Koefisien', key=abs, ascending=False)

for _, row in coef_df.iterrows():
    impact = "meningkatkan" if row['Koefisien'] > 0 else "menurunkan"
    print(f"  {row['Variabel']:12s}: {row['Koefisien']:7.1f} ({impact})")


Training Multiple Linear Regression Model...
Model Multiple Linear Regression berhasil dilatih!

PARAMETER MODEL:
- Intercept: 1248.32

Koefisien untuk setiap variabel:
 1. atemp       : 3488.0422 (Positif)
 2. temp        : 2097.2478 (Positif)
 3. windspeed   : -2080.5404 (Negatif)
 4. yr          : 2023.9975 (Positif)
 5. hum         : -865.4394 (Negatif)
 6. weathersit  : -632.8563 (Negatif)
 7. season      : 524.7225 (Positif)
 8. holiday     : -391.5508 (Negatif)
 9. workingday  : 160.8049 (Positif)
10. weekday     :  72.9370 (Positif)
11. mnth        : -38.4447 (Negatif)
Koefisien positif = meningkatkan penyewaan sepeda
Koefisien negatif = menurunkan penyewaan sepeda


In [ ]:
# ## 9. Test Model
# Prediksi dengan data testing
y_pred = model.predict(X_test)

# Hitung akurasi
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Hasil evaluasi model:")
print(f"R² Score: {r2:.4f} ({r2*100:.1f}%)")
print(f"RMSE: {rmse:.1f}")
print(f"MAE: {mae:.1f}")

# Interpretasi hasil
if r2 >= 0.8:
    rating = "Sangat Baik"
elif r2 >= 0.7:
    rating = "Baik"
elif r2 >= 0.6:
    rating = "Cukup"
else:
    rating = "Kurang"

print(f"\nKesimpulan: Model {rating} (akurasi {r2*100:.1f}%)")


In [ ]:
# ## 10. Visualisasi Hasil
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Plot 1: Prediksi vs Aktual
ax1.scatter(y_test, y_pred, alpha=0.6, color='steelblue', s=40)
ax1.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
ax1.set_xlabel('Nilai Aktual')
ax1.set_ylabel('Nilai Prediksi')
ax1.set_title('Perbandingan Prediksi vs Aktual')
ax1.grid(True, alpha=0.3)
ax1.text(0.05, 0.95, f'R² = {r2:.3f}', transform=ax1.transAxes,
         bbox=dict(boxstyle="round", facecolor="wheat"), fontsize=11)

# Plot 2: Error (Residual)
residuals = y_test - y_pred
ax2.scatter(y_pred, residuals, alpha=0.6, color='green', s=40)
ax2.axhline(y=0, color='red', linestyle='--', lw=2)
ax2.set_xlabel('Nilai Prediksi')
ax2.set_ylabel('Error (Aktual - Prediksi)')
ax2.set_title('Sebaran Error Model')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../OUTPUT/model_evaluation.png', dpi=300, bbox_inches='tight')
plt.show()

print("Penjelasan grafik:")
print("- Kiri: Jika titik mendekati garis merah, prediksi semakin akurat")
print("- Kanan: Error yang tersebar acak di sekitar 0 menunjukkan model baik")


In [ ]:
# ## 11. Analisis Pengaruh Variabel
# Buat grafik pengaruh setiap variabel
plt.figure(figsize=(12, 7))

# Warna berbeda untuk pengaruh positif dan negatif
colors = ['forestgreen' if x > 0 else 'crimson' for x in coef_df['Koefisien']]

bars = plt.barh(coef_df['Variabel'], coef_df['Koefisien'], color=colors, alpha=0.7)
plt.xlabel('Pengaruh terhadap Jumlah Penyewaan')
plt.title('Pengaruh Setiap Variabel terhadap Penyewaan Sepeda')
plt.grid(True, alpha=0.3, axis='x')
plt.axvline(x=0, color='black', linestyle='-', linewidth=0.8)

# Tambahkan nilai di ujung bar
for bar, val in zip(bars, coef_df['Koefisien']):
    plt.text(val + (50 if val > 0 else -50), bar.get_y() + bar.get_height()/2,
             f'{val:.0f}', ha='left' if val > 0 else 'right', va='center')

plt.tight_layout()
plt.savefig('../OUTPUT/feature_importance.png', dpi=300, bbox_inches='tight')
plt.show()

print("Interpretasi:")
print("- Hijau: Faktor yang meningkatkan penyewaan")
print("- Merah: Faktor yang menurunkan penyewaan")
print("- Semakin panjang bar, semakin besar pengaruhnya")


In [47]:
# ## 12. Contoh Prediksi
print("Contoh hasil prediksi untuk 5 data:")
print("-" * 45)

for i in range(5):
    actual = y_test.iloc[i]
    predicted = y_pred[i]
    error = abs(actual - predicted)
    error_pct = (error / actual) * 100

    print(f"Data ke-{i+1}:")
    print(f"  Aktual: {actual:.0f} penyewaan")
    print(f"  Prediksi: {predicted:.0f} penyewaan")
    print(f"  Selisih: {error:.0f} ({error_pct:.1f}%)")
    print()


Contoh Prediksi untuk Data Baru:
Sampel 1:
  Nilai Aktual: 6606 penyewaan
  Nilai Prediksi: 6220 penyewaan
  Error: 386 (5.8%)

Sampel 2:
  Nilai Aktual: 1550 penyewaan
  Nilai Prediksi: 1572 penyewaan
  Error: 22 (1.4%)

Sampel 3:
  Nilai Aktual: 3747 penyewaan
  Nilai Prediksi: 3044 penyewaan
  Error: 703 (18.8%)

Sampel 4:
  Nilai Aktual: 6041 penyewaan
  Nilai Prediksi: 4307 penyewaan
  Error: 1734 (28.7%)

Sampel 5:
  Nilai Aktual: 7538 penyewaan
  Nilai Prediksi: 6766 penyewaan
  Error: 772 (10.2%)

Contoh prediksi menunjukkan variasi akurasi model pada data testing.


In [ ]:
# ## Kesimpulan
print("=" * 50)
print("KESIMPULAN ANALISIS")
print("=" * 50)

# Faktor paling berpengaruh
top_positive = coef_df[coef_df['Koefisien'] > 0].head(3)
top_negative = coef_df[coef_df['Koefisien'] < 0].head(3)

print(f"1. Akurasi Model: {r2*100:.1f}% - {rating}")
print(f"2. RMSE: {rmse:.1f} (rata-rata error)")
print()
print("3. Faktor yang MENINGKATKAN penyewaan:")
for _, row in top_positive.iterrows():
    print(f"   - {row['Variabel']}: +{row['Koefisien']:.0f}")
print()
print("4. Faktor yang MENURUNKAN penyewaan:")
for _, row in top_negative.iterrows():
    print(f"   - {row['Variabel']}: {row['Koefisien']:.0f}")
print()
print("5. Insight Bisnis:")
print("   - Suhu yang nyaman (atemp) paling berpengaruh positif")
print("   - Angin kencang (windspeed) paling berpengaruh negatif")
print("   - Model ini bisa dipakai untuk prediksi permintaan sepeda")
print("=" * 50)
